# Deal or No Deal

## Dependency

In [104]:
import os
import glob
import collections
import numpy as np
import pandas as pd
import matplotlib
from pathlib import Path


import torch
from torch import nn
import torch.tensor
import torch.optim as optim
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.mode.chained_assignment = None

# Configs

In [105]:
DATA_DIR = '.'
REGION = 'us'
INSTRUMENT = 'unh'

raw_data_removed_features_set = ['open', 'high', 'low', 'close']
raw_data_added_features_set = ['volume']
raw_data_features_set = raw_data_removed_features_set + raw_data_added_features_set

raw_data_features_set = ['open', 'high', 'low', 'close', 'volume']
raw_target = 'close'

features_set = ['volume',
                'high_open_delta',
                'low_close_delta',
                'high_low_delta',
                'open_close_delta',
                'high_close_delta',
                'low_open_delta']
target = 'close_delta'

# Helpers

### Market Data

In [143]:
def fetch_instrument_file(instrument, region, base_dir):
    cwd = os.getcwd()
    os.chdir(base_dir)
    
    instrument_file_list = result = list(Path(".").rglob(f"*{instrument}*.txt"))
    if not instrument_file_list:
        os.chdir(cwd)
        raise Exception(f'Cannot find file for instrument {instrument}')
    if len(instrument_file_list) > 1:
        os.chdir(cwd)
        raise Exception(f'Found multiple file for instrument {instrument}')
        
    os.chdir(cwd)
    return str(os.path.join(os.getcwd(), instrument_file_list[0]))

instrument_data_file = fetch_instrument_file(INSTRUMENT, REGION, DATA_DIR)
instrument_data_file

'/Users/myousse3/data/dev/personal/be-mil/data/daily/us/nyse stocks/2/unh.us.txt'

# Data Preparation 

In [144]:
df = pd.read_csv(instrument_data_file)
df.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'openint']
df.date = pd.to_datetime(df.date, format='%Y%m%d')
df

,date,open,high,low,close,volume,openint
0,1990-03-26,0.25264,0.25264,0.25264,0.25264,473990,0
1,1990-03-27,0.25264,0.26996,0.25264,0.26996,4493732,0
2,1990-03-28,0.26996,0.26996,0.26119,0.26119,1793083,0
3,1990-03-29,0.26996,0.26996,0.26119,0.26119,617291,0
4,1990-03-30,0.26996,0.26996,0.26119,0.26119,2017221,0
...,...,...,...,...,...,...,...
7510,2020-01-16,298.00000,300.99000,295.93000,300.74000,3710344,0
7511,2020-01-17,300.01000,300.70000,295.88000,298.47000,4632160,0
7512,2020-01-21,296.89000,302.54000,296.01000,300.53000,4963132,0
7513,2020-01-22,301.99000,302.13000,297.35000,300.59000,3117568,0


In [145]:
df = df[raw_data_features_set]
df

,open,high,low,close,volume
0,0.25264,0.25264,0.25264,0.25264,473990
1,0.25264,0.26996,0.25264,0.26996,4493732
2,0.26996,0.26996,0.26119,0.26119,1793083
3,0.26996,0.26996,0.26119,0.26119,617291
4,0.26996,0.26996,0.26119,0.26119,2017221
...,...,...,...,...,...
7510,298.00000,300.99000,295.93000,300.74000,3710344
7511,300.01000,300.70000,295.88000,298.47000,4632160
7512,296.89000,302.54000,296.01000,300.53000,4963132
7513,301.99000,302.13000,297.35000,300.59000,3117568


In [146]:
df['high_open_delta'] = df['high'] - df['open']
df['low_close_delta'] = df['low'] - df['close']

df['high_low_delta'] = df['high'] - df['low']
df['open_close_delta'] = df['open'] - df['close']

df['high_close_delta'] = df['high'] - df['close']
df['low_open_delta'] = df['low'] - df['open']

df[f'{raw_target}_t1'] = df[raw_target].shift(-1)
df[f'{raw_target}_delta'] = df[f'{raw_target}_t1'] - df[raw_target]

df = df[:-1]
df

,open,high,low,close,volume,high_open_delta,low_close_delta,high_low_delta,open_close_delta,high_close_delta,low_open_delta,close_t1,close_delta
0,0.25264,0.25264,0.25264,0.25264,473990,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.26996,0.01732
1,0.25264,0.26996,0.25264,0.26996,4493732,0.01732,-0.01732,0.01732,-0.01732,0.00000,0.00000,0.26119,-0.00877
2,0.26996,0.26996,0.26119,0.26119,1793083,0.00000,0.00000,0.00877,0.00877,0.00877,-0.00877,0.26119,0.00000
3,0.26996,0.26996,0.26119,0.26119,617291,0.00000,0.00000,0.00877,0.00877,0.00877,-0.00877,0.26119,0.00000
4,0.26996,0.26996,0.26119,0.26119,2017221,0.00000,0.00000,0.00877,0.00877,0.00877,-0.00877,0.26996,0.00877
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7509,289.02000,299.64000,289.00000,296.41000,6081615,10.62000,-7.41000,10.64000,-7.39000,3.23000,-0.02000,300.74000,4.33000
7510,298.00000,300.99000,295.93000,300.74000,3710344,2.99000,-4.81000,5.06000,-2.74000,0.25000,-2.07000,298.47000,-2.27000
7511,300.01000,300.70000,295.88000,298.47000,4632160,0.69000,-2.59000,4.82000,1.54000,2.23000,-4.13000,300.53000,2.06000
7512,296.89000,302.54000,296.01000,300.53000,4963132,5.65000,-4.52000,6.53000,-3.64000,2.01000,-0.88000,300.59000,0.06000


In [147]:
df.drop([f'{raw_target}_t1'], axis=1, inplace=True)
df.drop(raw_data_removed_features_set, axis=1, inplace=True)
df

,volume,high_open_delta,low_close_delta,high_low_delta,open_close_delta,high_close_delta,low_open_delta,close_delta
0,473990,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01732
1,4493732,0.01732,-0.01732,0.01732,-0.01732,0.00000,0.00000,-0.00877
2,1793083,0.00000,0.00000,0.00877,0.00877,0.00877,-0.00877,0.00000
3,617291,0.00000,0.00000,0.00877,0.00877,0.00877,-0.00877,0.00000
4,2017221,0.00000,0.00000,0.00877,0.00877,0.00877,-0.00877,0.00877
...,...,...,...,...,...,...,...,...
7509,6081615,10.62000,-7.41000,10.64000,-7.39000,3.23000,-0.02000,4.33000
7510,3710344,2.99000,-4.81000,5.06000,-2.74000,0.25000,-2.07000,-2.27000
7511,4632160,0.69000,-2.59000,4.82000,1.54000,2.23000,-4.13000,2.06000
7512,4963132,5.65000,-4.52000,6.53000,-3.64000,2.01000,-0.88000,0.06000


In [148]:
df_summary = df.describe()
df_summary

,volume,high_open_delta,low_close_delta,high_low_delta,open_close_delta,high_close_delta,low_open_delta,close_delta
count,7.514000e+03,7514.000000,7514.000000,7514.000000,7514.000000,7514.000000,7514.000000,7514.000000
mean,8.555819e+06,0.492946,-0.521362,1.005653,-0.008654,0.484292,-0.512708,0.039820
std,7.241209e+06,0.901377,0.963271,1.496808,1.064109,0.924253,1.048638,1.184307
min,0.000000e+00,0.000000,-14.530000,0.000000,-13.240000,0.000000,-21.910000,-12.690000
25%,4.464686e+06,0.043800,-0.564750,0.157000,-0.199675,0.052300,-0.556000,-0.167750
50%,6.791788e+06,0.191000,-0.194500,0.543500,0.000000,0.180000,-0.176000,0.008660
75%,1.038458e+07,0.564000,-0.053000,1.170000,0.163000,0.540000,-0.043415,0.235000
max,1.858094e+08,19.380000,0.000000,23.910000,16.830000,16.830000,0.000000,17.930000


In [149]:
df['high_open_delta_std2'] = (df['high_open_delta'] - df_summary['high_open_delta']['mean']) / df_summary['high_open_delta']['std']
df[f'high_open_delta_std1'] = StandardScaler().fit_transform(df[[f'high_open_delta']])

df

# for feature in features_set:
#     df[feature] = StandardScaler().fit_transform(df[[feature]])
# df[f'{raw_target}_delta'] = StandardScaler().fit_transform(df[[f'{raw_target}_delta']])



,volume,high_open_delta,low_close_delta,high_low_delta,open_close_delta,high_close_delta,low_open_delta,close_delta,high_open_delta_std2,high_open_delta_std1
0,473990,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01732,-0.546880,-0.546917
1,4493732,0.01732,-0.01732,0.01732,-0.01732,0.00000,0.00000,-0.00877,-0.527665,-0.527701
2,1793083,0.00000,0.00000,0.00877,0.00877,0.00877,-0.00877,0.00000,-0.546880,-0.546917
3,617291,0.00000,0.00000,0.00877,0.00877,0.00877,-0.00877,0.00000,-0.546880,-0.546917
4,2017221,0.00000,0.00000,0.00877,0.00877,0.00877,-0.00877,0.00877,-0.546880,-0.546917
...,...,...,...,...,...,...,...,...,...,...
7509,6081615,10.62000,-7.41000,10.64000,-7.39000,3.23000,-0.02000,4.33000,11.235092,11.235840
7510,3710344,2.99000,-4.81000,5.06000,-2.74000,0.25000,-2.07000,-2.27000,2.770266,2.770451
7511,4632160,0.69000,-2.59000,4.82000,1.54000,2.23000,-4.13000,2.06000,0.218615,0.218629
7512,4963132,5.65000,-4.52000,6.53000,-3.64000,2.01000,-0.88000,0.06000,5.721307,5.721687


In [119]:
for feature in features_set:
    df[feature] = StandardScaler().inverse_transform(df[[feature]])
df[f'{raw_target}_delta'] = StandardScaler().inverse_transform(df[[f'{raw_target}_delta']])

df


NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [47]:
df.describe()

,volume,high_open_delta,low_close_delta,high_low_delta,open_close_delta,high_close_delta,low_open_delta,close_delta
count,7514.000000,7514.000000,7514.000000,7514.000000,7514.000000,7514.000000,7514.000000,7514.000000
mean,0.000099,-0.000011,0.000316,-0.000260,-0.000070,-0.000092,0.000361,0.000000
std,1.000096,1.000133,0.999758,0.999879,1.000115,1.000102,0.999643,1.000067
min,-1.181560,-0.546964,-14.538933,-0.672044,-12.435694,-0.524127,-20.397184,-10.749462
25%,-0.564935,-0.498366,-0.044716,-0.567167,-0.179603,-0.467535,-0.040908,-0.175279
50%,-0.243534,-0.335038,0.339558,-0.308982,0.008063,-0.329355,0.321337,-0.026312
75%,0.252672,0.078828,0.486418,0.109525,0.161261,0.060188,0.447727,0.164816
max,24.480908,20.956323,0.541425,15.300016,15.825921,17.687030,0.489114,15.107033


In [46]:

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


def _get_ops(op):
    if op == 'avg':
        return np.mean

    raise Exception(f'Unknown operation {op}')


def _aggregate_collection(lst, op):
    if len(lst) == 0:
        return {}
    elif len(lst) == 1:
        return lst[0]
    else:
        aggrgated_values_dict = {}
        for entry in lst:
            for key, value in entry.items():
                if key in aggrgated_values_dict:
                    aggrgated_values_dict[key].append(value)
                else:
                    aggrgated_values_dict[key] = [value]

        final_aggrgated_values_dict = {}
        for key, values in aggrgated_values_dict.items():
            values = [e for e in values if e]
            final_aggrgated_values_dict[key] = op(values)

        return final_aggrgated_values_dict


def _day_aggregator_handler(dataset, **kwargs):
    length = kwargs['length']
    return dataset[0:length], dataset[length:]


def _week_aggregator_handler(dataset, **kwargs):
    return _universal_aggregator_handler(dataset, 5, **kwargs)


def _month_aggregator_handler(dataset, **kwargs):
    return _universal_aggregator_handler(dataset, 21, **kwargs)


def _quarterly_aggregator_handler(dataset, **kwargs):
    return _universal_aggregator_handler(dataset, 21 * 3, **kwargs)


def _yearly_aggregator_handler(dataset, **kwargs):
    return _universal_aggregator_handler(dataset, 21 * 12, **kwargs)


def _universal_aggregator_handler(dataset, period_range, **kwargs):
    length = kwargs['length']
    ops = _get_ops(kwargs['op'])

    if length == -1:
        return [_aggregate_collection(dataset, ops)], []

    else:
        aggregated_dataset = []

        raw_dataset = dataset[0:length]
        raw_chunked_dataset = chunks(raw_dataset, period_range)
        for week in raw_chunked_dataset:
            aggregated_dataset.append(_aggregate_collection(week, ops))

        return aggregated_dataset, dataset[length:]


def features_aggregator(dataset, features_descriptors):
    aggregated_dataset = []
    for feature_set in features_descriptors:
        index, length, handler, params = feature_set['index'], feature_set['length'], feature_set['handler'], feature_set['params']

        if length == -1:
            period_range = len(dataset)
            step_aggregated_dataset, dataset = handler(dataset, period_range, **{
                'index': index,
                'length': length,

                **params
            })
        else:
            step_aggregated_dataset, dataset = handler(dataset, **{
                'index': index,
                'length': length,

                **params
            })

        if len(step_aggregated_dataset) > 0:
            aggregated_dataset += step_aggregated_dataset

        if len(dataset) == 0:
            return aggregated_dataset

    return aggregated_dataset


def train_dataset_generator(df, shift_range=21, repeat_out=2):
    _features_descriptors = [
        {'index': 0, 'length': 21 * 12 * 1, 'handler': _day_aggregator_handler, 'params': {}},
        {'index': 1, 'length': 21 * 12 * 6, 'handler': _week_aggregator_handler, 'params': {'op': 'avg'}},
        {'index': 2, 'length': 21 * 12 * 6, 'handler': _month_aggregator_handler, 'params': {'op': 'avg'}},
        {'index': 3, 'length': 21 * 12 * 6, 'handler': _quarterly_aggregator_handler, 'params': {'op': 'avg'}},
        {'index': 4, 'length': 21 * 12 * 6, 'handler': _yearly_aggregator_handler, 'params': {'op': 'avg'}},
        {'index': 5, 'length': -1, 'handler': _universal_aggregator_handler, 'params': {'op': 'avg'}},
    ]

    dataset = list(df.T.to_dict().values())
    train_dataset = []
    while repeat_out > 0:
        train_subset = features_aggregator(dataset, _features_descriptors)
        train_dataset.append(train_subset)
        dataset = dataset[shift_range:]
        repeat_out -= 1

    return train_dataset


train_dataset = train_dataset_generator(df, shift_range=5, repeat_out=100)
len(train_dataset[0])
train_dataset[0][0]

658

{'volume': -1.1160968214833211,
 'high_open_delta': -0.5469643632708262,
 'low_close_delta': 0.5414254168685192,
 'high_low_delta': -0.6720439003895374,
 'open_close_delta': 0.008063475621591872,
 'high_close_delta': -0.524126754926294,
 'low_open_delta': 0.4891136456494993,
 'close_delta': -0.018999697295633926}

In [73]:
def pad_dataset_sequence(dataset):
    if len(dataset) == 0:
        return dataset

    max_sequence = max([len(e) for e in dataset])

    input_example_dict = dataset[0][0]
    pad_example_dict = {k: 0.0 for k, _ in input_example_dict.items()}

    for example in dataset:
        residual_sequence_length = max_sequence - len(example)
        if residual_sequence_length > 0:
            for e in [pad_example_dict] * residual_sequence_length:
                example.append(e)

    return dataset


print(collections.Counter([len(e) for e in train_dataset]))
train_dataset = pad_dataset_sequence(train_dataset)
print(collections.Counter([len(e) for e in train_dataset]))

Counter({658: 243, 657: 51, 654: 51, 652: 51, 656: 50, 655: 50, 653: 50, 650: 13, 648: 13, 647: 13, 645: 13, 643: 13, 642: 13, 640: 13, 638: 13, 637: 13, 635: 13, 633: 13, 632: 13, 630: 13, 628: 13, 651: 12, 649: 12, 646: 12, 644: 12, 641: 12, 639: 12, 636: 12, 634: 12, 631: 12, 629: 12, 625: 5, 620: 5, 615: 5, 610: 5, 605: 5, 600: 5, 595: 5, 627: 4, 626: 4, 624: 4, 623: 4, 622: 4, 621: 4, 619: 4, 618: 4, 617: 4, 616: 4, 614: 4, 613: 4, 612: 4, 611: 4, 609: 4, 608: 4, 607: 4, 606: 4, 604: 4, 603: 4, 602: 4, 601: 4, 599: 4, 598: 4, 597: 4, 596: 4, 594: 4, 593: 4, 592: 4, 591: 1})
Counter({658: 1000})


# Modeling

In [74]:
# Hyper-params

param_epochs = 2
param_batch_size = 1

param_input_size = 7
param_sequence_size = 658
param_layers_size = 1
param_hidden_size = 1
param_dropout = 0.05
param_dense_1 = 1024
param_dense_2 = 512
param_dense_3 = 128
param_output_size = 1

gpu_enabled = torch.cuda.is_available()
device = torch.device("cuda") if gpu_enabled else torch.device("cpu")

param_lr = 0.01

In [75]:
class SingleInstrumentPredictorRNN(nn.Module):
    
    def __init__(self):
        super(SingleInstrumentPredictorRNN, self).__init__()
        
        self._rnn = nn.RNN(param_input_size, param_hidden_size, param_layers_size, batch_first=True, dropout=param_dropout)
        
        self._fc_1 = nn.Linear(param_dense_1, param_dense_2)
        self._fc_2 = nn.Linear(param_dense_2, param_dense_3)
        self._fc_3 = nn.Linear(param_dense_3, param_output_size)

    def forward(self, input):
        batch_size = input.size(0)
        

        hidden = self.init_hidden(batch_size)
        out, hidden = self._rnn(input, hidden)
        
        out = out.view(-1, self.hidden_dim)
        
        out = F.relu(self._fc_1(out))
        out = F.relu(self._fc_2(out))
        out = F.relu(self._fc_3(out))
  
        return out, hidden
    

    def init_hidden(self, batch_size):
        hidden = torch.zeros(param_layers_size, batch_size, param_hidden_size)
        return hidden


## Training

In [76]:
model = SingleInstrumentPredictorRNN()

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=param_lr)

if gpu_enabled:
    model.cuda()
    print('GPU Enabled Model')
else:
    print('GPU Disabled Model')

GPU Disabled Model


/Users/myousse3/anaconda3/envs/dealnodeal/lib/python3.6/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.05 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [80]:
def tensorify_example(example):
    example_df = pd.DataFrame(example)

    features_tensor = torch.tensor(example_df[features_set].values)
    features_tensor = features_tensor.unsqueeze(0)

    target_tensor = torch.tensor(example_df[target].values)
    target_tensor = target_tensor.view(1, -1)

    # features_tensor.size() # torch.Size([1, 658, 7])
    # target_tensor.size() # torch.Size([1, 658])
    return features_tensor, target_tensor

def batch_tensorify(examples_batch):
    features_tensors_list = [tensorify_example(example)[0] for example in examples_batch]
    target_tensors_list = [tensorify_example(example)[1] for example in examples_batch]
    
    return torch.cat(features_tensors_list, 0), torch.cat(target_tensors_list, 0)



exmaple_1 = train_dataset[0]
exmaple_2 = train_dataset[1]
batch_features, batch_target = batch_tensorify([exmaple_1, exmaple_2])

batch_features.size()
batch_features

batch_target.size()
batch_target

torch.Size([2, 658, 7])

tensor([[[-1.1161,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.5609,  0.0173, -0.0173,  ..., -0.0173,  0.0000,  0.0000],
         [-0.9339,  0.0000,  0.0000,  ...,  0.0088,  0.0088, -0.0088],
         ...,
         [-0.3977,  0.6075, -0.5324,  ..., -0.0717,  0.5318, -0.4738],
         [-0.5817,  0.7584, -0.7563,  ..., -0.0850,  0.6606, -0.6788],
         [-0.6564,  1.6468, -1.7875,  ..., -0.0047,  1.6120, -1.8084]],

        [[-0.7928,  0.0088, -0.0173,  ..., -0.0088,  0.0000, -0.0086],
         [-0.7355,  0.0087, -0.0088,  ...,  0.0000,  0.0087, -0.0088],
         [-0.5467,  0.0000,  0.0000,  ...,  0.0087,  0.0087, -0.0087],
         ...,
         [-0.4187,  0.5907, -0.5240,  ..., -0.0587,  0.5284, -0.4787],
         [-0.5833,  0.7698, -0.7642,  ..., -0.0727,  0.6811, -0.6993],
         [-0.6566,  1.6488, -1.7920,  ..., -0.0079,  1.6120, -1.8097]]],
       dtype=torch.float64)

torch.Size([2, 658])

tensor([[ 0.0173, -0.0088,  0.0000,  ...,  0.0919,  0.1413,  0.1570],
        [ 0.0000,  0.0000,  0.0087,  ...,  0.0770,  0.1358,  0.1579]],
       dtype=torch.float64)

In [82]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]
    

for epoch in range(1, param_epochs + 1):
    
    loss = 0
    for batch_examples in batch(train_dataset, param_batch_size):
        
        batch_features, batch_target = batch_tensorify(batch_examples)
        if gpu_enabled:
            input_batch = input_batch.cuda()
            labels_batch = labels_batch.cuda()
            
        optimizer.zero_grad()
        
        batch_features.to(device)
        batch_target.to(device)
        
        output, hidden = model(batch_features)       
        
        loss = criterion(output, batch_target)
        loss.backward()
        optimizer.step()

    print(f'Epoch: {epoch}/{epochs} ............. Loss: {loss.item()}')

tensor([[[-1.1161,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.5609,  0.0173, -0.0173,  ..., -0.0173,  0.0000,  0.0000],
         [-0.9339,  0.0000,  0.0000,  ...,  0.0088,  0.0088, -0.0088],
         ...,
         [-0.3977,  0.6075, -0.5324,  ..., -0.0717,  0.5318, -0.4738],
         [-0.5817,  0.7584, -0.7563,  ..., -0.0850,  0.6606, -0.6788],
         [-0.6564,  1.6468, -1.7875,  ..., -0.0047,  1.6120, -1.8084]]],
       dtype=torch.float64)

tensor([[ 0.0173, -0.0088,  0.0000,  0.0000,  0.0088,  0.0000,  0.0000,  0.0087,
          0.0089, -0.0089, -0.0087,  0.0087,  0.0089,  0.0000,  0.0000, -0.0089,
          0.0000,  0.0000, -0.0087,  0.0000,  0.0087,  0.0000, -0.0087,  0.0263,
          0.0085,  0.0089,  0.0000,  0.0174,  0.0175,  0.0088, -0.0088,  0.0000,
          0.0000,  0.0088,  0.0000,  0.0000,  0.0086, -0.0086,  0.0173,  0.0175,
          0.0086,  0.0087, -0.0087,  0.0087,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0176, -0.0087, -0.0089, -0.0261,  0.0000,  0.0174,  0.0175,  0.0000,
         -0.0089,  0.0000,  0.0089,  0.0087,  0.0000,  0.0174,  0.0173, -0.0087,
         -0.0086,  0.0086,  0.0173,  0.0000,  0.0088,  0.0000,  0.0000, -0.0088,
         -0.0173,  0.0261,  0.0349,  0.0086,  0.0000, -0.0174, -0.0086, -0.0264,
          0.0088, -0.0175,  0.0350,  0.0174,  0.0000, -0.0174,  0.0174,  0.0000,
         -0.0174, -0.0088, -0.0435, -0.0087,  0.0347, -0.0173,  0.0349, -0.0087,
          0.0176, -0.0176,  

RuntimeError: Expected object of scalar type Double but got scalar type Float for argument #2 'mat2' in call to _th_mm

In [136]:
# Evaludation